In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn.datasets

/usr/lib64/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
iris = sklearn.datasets.load_iris()
x, y = iris.data, iris.target
y = pd.get_dummies(y).values

In [10]:
def model(x, y):
    a1 = tf.layers.dense(x, 128, activation=tf.nn.tanh, use_bias=True)
    a2 = tf.layers.dense(a1, 32, activation=tf.nn.tanh, use_bias=True)
    y_hat = tf.layers.dense(a2, 3, activation=tf.sigmoid, use_bias=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_hat))
    correct_pred = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y, 1), name='correct_pred')  
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    return loss, accuracy

In [11]:
def average_gradients_my(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        grads = []
        for _, g in grad_and_vars:
            expanded_g = tf.expand_dims(g, 0)
            grads.append(expanded_g)
        # Average over the 'tower' dimension.
        grad = tf.concat(axis=0, values=grads)
        grad = tf.reduce_mean(grad, 0)
        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [12]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
config.gpu_options.per_process_gpu_memory_fraction = 0.5 
config.log_device_placement = True

# 多GPU

In [17]:
with tf.device('/cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((x, y)).repeat(5000).shuffle(buffer_size=128).batch(64)
    iter_ds = dataset.make_one_shot_iterator()
    opt = tf.train.GradientDescentOptimizer(0.01)
    grads = []
    for i in range(4):
        with tf.device('/gpu:%d' % i):
            print 'on gpu:' + str(i)
            xx, yy = iter_ds.get_next()
            loss, accuracy = model(xx, yy)
            grad_var = opt.compute_gradients(loss)
            grads.append(grad_var)
    avg_grad = average_gradients_my(grads)
    train_op = opt.apply_gradients(avg_grad)
    # 
    sess = tf.Session(config=config)
    sess.run(tf.global_variables_initializer())
    try:
        for i in range(10000):
            _loss,_accuracy, _ = sess.run([loss, accuracy, train_op])
            if (i+1) % 500 == 0:
                print '%d : %.4f : %.4f ' % (i + 1, _loss, _accuracy)
    except:
        print 'end'

on gpu:0
on gpu:1
on gpu:2
on gpu:3
500 : 1.0794 : 0.4062 
1000 : 1.0796 : 0.4219 
1500 : 1.1235 : 0.2656 
2000 : 1.0957 : 0.4375 
2500 : 1.2065 : 0.2812 
3000 : 1.1706 : 0.1875 
3500 : 1.0843 : 0.4219 
4000 : 1.1324 : 0.3281 
4500 : 1.0946 : 0.3438 
5000 : 1.0756 : 0.4062 
5500 : 1.1791 : 0.2500 
6000 : 1.1340 : 0.2656 
6500 : 1.0550 : 0.4062 
7000 : 1.1651 : 0.3125 
7500 : 1.1212 : 0.3281 
8000 : 1.1056 : 0.3594 
8500 : 1.1605 : 0.2969 
9000 : 1.1322 : 0.2500 
9500 : 1.0631 : 0.4531 
10000 : 1.1762 : 0.2344 


# CPU

In [16]:
with tf.device('/cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((x, y)).repeat(5000).shuffle(buffer_size=128).batch(64)
    iter_ds = dataset.make_one_shot_iterator()
    xx, yy = iter_ds.get_next()
    loss, accuracy = model(xx, yy)
    train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    # 
    sess = tf.Session(config=config)
    sess.run(tf.global_variables_initializer())
    try:
        for i in range(10000):
            _loss,_accuracy, _ = sess.run([loss, accuracy, train_op])
            if (i+1) % 500 == 0:
                print '%d : %.4f : %.4f ' % (i + 1, _loss, _accuracy)
    except:
        print 'end'

500 : 0.7370 : 0.5625 
1000 : 0.7016 : 0.6094 
1500 : 0.6433 : 0.8438 
2000 : 0.6897 : 0.7188 
2500 : 0.6622 : 0.8281 
3000 : 0.6761 : 0.8125 
3500 : 0.7066 : 0.7969 
4000 : 0.6566 : 0.8438 
4500 : 0.6697 : 0.9062 
5000 : 0.7004 : 0.7969 
5500 : 0.6920 : 0.8438 
6000 : 0.6334 : 0.9219 
6500 : 0.6752 : 0.9219 
7000 : 0.6452 : 0.9375 
7500 : 0.6513 : 0.9062 
8000 : 0.6893 : 0.9219 
8500 : 0.6657 : 0.9062 
9000 : 0.6359 : 0.9375 
9500 : 0.6575 : 0.9375 
10000 : 0.6740 : 0.9062 
